In [1]:
import numpy as np
import pandas as pd
import re
import string

import nltk
from bangla_stemmer.stemmer import stemmer
from nltk.tokenize import WhitespaceTokenizer
from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

import matplotlib.pyplot as plt

In [2]:
def read_files(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        lines = []
        for line in file:
            lines.append(line)
        file.close()
    return lines

In [3]:
negative_lines = read_files(filename = "final_neg_merged.txt")
positive_lines = read_files(filename = "final_pos_merged.txt")

In [4]:
def strip_lines(lines):
    modified_lines = []
    for i in range(len(lines)):
        temp = lines[i]
        temp = temp.lstrip().rstrip("\n")
        modified_lines.append(temp)
    return modified_lines

In [5]:
stripped_negative = strip_lines(negative_lines)
stripped_positive = strip_lines(positive_lines)

In [6]:
stripped_negative[1:100]

['সাধারণ মানুষ টিভি চ্যানেল গুলো ছুড়ে ফেলে দিয়ে ইউটিউবে নাটক দেখে কারণ  টিভি চ্যানেল গুলোর বিরক্তিকর এ্যাডগুলোর জন্য। এখন এখানেও সেই ভাইরাস / রোগ ঢুকে গেছে। এই কম্পিটিশন এর যুগে ইউটিউব কতৃপক্ষ যদি বিজ্ঞাপন দেয়ার বিষয়ে আরও সচেতন এবং সুদূরপ্রসারী চিন্তা ভাবনা না করেন এক সময় আপনারাও কালের গর্ভে হারিয়ে যাবেন।',
 'স্টুডেন্ট ওরে একটা থাপ্পড় মারো',
 'নাটকটিতে কথা থেকে মিউজিকের সাউন্ডটা বেশি হয়ে গেছে,যে কারনে কথাগুলো অনেক সময় ক্লিয়ার শোনা যায় না।',
 'পাকনা মেয়াটা একটা বাজে',
 'ধূস পুরো ফালতু. স্টোরি খুব দুর্বল',
 'কথার চেয়ে মিউজিকের আওয়াজ বেশি',
 'আমি আফরান নিশোর  অনেক বড় ফ্যান কিন্তু ইদানিং সে অনেক বাজে নাটক করছে তার উচিৎ দেখে শুনে কাজ করা',
 'নাটকের শেষাংশে সনাতনি কায়দা অবলম্বন করা হয়েছে, যা আমার কাছে ভালো লাগে নাই, চাই নতুনত্ব!',
 'শেষ টা ভাল লাগলো না',
 'vai maybe science বানান টা ভুল হয়েছে।',
 'স্টুডেন্ট মেয়েটা নাটকটাকে বিরক্তিকর  বানিয়েছে..',
 'স্টুডেন্ট টারে ইচ্ছে করে কসায় একটা টাপ্পর মারি।',
 'এই স্টুডেন্ট এর ক্যারেক্টার টা পুরা নাটকের কমপ্লেক্সান টা নষ্ট করে দিসে।

In [7]:
def remove_punc(lines):
    punctuations = '''!()-[]।{};:'"\,<>./?@#$%^&*_~'''
    punch_remove_line = []
    # remove punctuation from the string
    for i in range(len(lines)):
        no_punct = ""
        for char in lines[i]:
            if char not in punctuations:
                no_punct = no_punct + char
        punch_remove_line.append(no_punct)
    return punch_remove_line

In [8]:
positive_lines = remove_punc(stripped_positive)
negative_lines = remove_punc(stripped_negative)

In [9]:
def remove_english_words(lines):
    sentences = []
    for i in range(len(lines)):
        temp = lines[i]
        lst = temp.split()
        flag = 0
        for word in lst:
            x = re.search(r"^[a-zA-Z0-9]", word)
            if x:
                flag += 1
        if flag == 0:
            sentences.append(temp)
    return sentences

In [10]:
positive_lines = remove_english_words(positive_lines)
negative_lines = remove_english_words(negative_lines)

In [11]:
print(len(positive_lines))
print(len(negative_lines))

11014
4705


In [12]:
def create_dataset_csv(lines, filename, number):
    df = pd.DataFrame(lines, columns=["text"])
    if number == 0:
        label = np.zeros(len(lines))
        df['label'] = label
    elif number == 1:
        label = np.ones(len(lines))
        df['label'] = label
    df.to_csv(filename, index=False)

In [13]:
create_dataset_csv(positive_lines, "positive_final.csv", 1)
create_dataset_csv(negative_lines, "negative_final.csv", 0)

In [14]:
df_pos = pd.read_csv("positive_final.csv")
df_neg = pd.read_csv("negative_final.csv")

In [15]:
df_pos_new = df_pos[:7000]

In [16]:
# append both pos and neg
pos_neg = df_pos_new.append(df_neg)

In [17]:
pos_neg.to_csv("all_pos_neg_final.csv", index = False)

In [18]:
finaldf = pd.read_csv("all_pos_neg_final.csv")

In [19]:
finaldf.shape[0]

11705

In [20]:
np.random.seed(42)
finaldf = finaldf.sample(frac = 1) # shuffle
finaldf

,text,label
9294,মোশাররফ করিমের সব খোর নাটক গুলাই বাজে,0.0
6392,ঈদের সেরা নাটক আমার দেখা অনেক ভালো লাগলো নাটকট...,1.0
2955,নাটক এর শুরু টা ভালো ছিলো শেষ টা কস্টের ...,1.0
9309,বালের নাটক,0.0
11134,ছেলে মেয়ে নষ্ট হওয়ার মত একটা নাটক,0.0
...,...,...
11284,ধ্যাত বাল,0.0
5191,চমৎকার গল্প খুব ভালো লাগলো,1.0
5390,শওি নাটকটা অনেক সুন্দর,1.0
860,ভালোবাসা বুজি এমনই হয় নিজে তিলে তিলে শেষ হয়ে য...,1.0


In [21]:
finaldf.to_csv('final_shuffled_dataset.csv', index = False)

In [20]:
# data preprocess and stuff

In [21]:
stop_words = []
with open('bangla_sw.txt','r', encoding='utf-8') as file:    
    for line in file:      
        for word in line.split():         
            stop_words.append(word)
print(type(stop_words))
print(len(stop_words))

<class 'list'>
398


In [22]:
X = list()
Y = list()
for index, row in finaldf.iterrows():
    X.append(getattr(row, "text"))
    Y.append(getattr(row, "label"))

In [23]:
# yslist = np.squeeze(Y).tolist()
# print(Y)

In [24]:
# print(yslist)

In [25]:
def process_senteence(sentence):
    stmr = stemmer.BanglaStemmer()
    
    tokenizer = WhitespaceTokenizer()
    tokens = tokenizer.tokenize(sentence)

    sentence_clean = []
    for word in tokens:
        if (word not in stop_words):
            stem_word = stmr.stem(word)  # stemming word
            sentence_clean.append(stem_word)

    return sentence_clean


def build_freqs(sentences, ys):
    yslist = ys
    freqs = {}
    for y, sentence in zip(yslist, sentences):
        for word in process_senteence(sentence):
            # print(word)
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1
    return freqs

In [26]:
x_train,x_test,y_train,y_test= train_test_split(X,Y,test_size=0.2)

In [27]:
y_train

[0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0

In [28]:
freqs = build_freqs(x_train, y_train)

print("type(freqs) = " + str(type(freqs)))
print("len(freqs) = " + str(len(freqs.keys())))

applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied third rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
appl

applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
appli

applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applie

applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied third rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied second rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied third rules..
a

applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..


...

applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied third rules..
applied second rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied third rules..
applied fourth rules..
applied second rules..
applied fourth rules..
appli

applied fourth rules..
applied first rules..
applied second rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules

applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..

applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
appl

applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules.

In [29]:
freqs

{('ফালতু', 0.0): 688,
 ('এক', 0.0): 521,
 ('নাটুকযত', 0.0): 1,
 ('আজাইরা', 0.0): 20,
 ('গুহুর', 0.0): 1,
 ('দেইতে', 0.0): 1,
 ('চাই', 0.0): 43,
 ('দিয়েগুহুর', 0.0): 1,
 ('এটা', 1.0): 21,
 ('নাম', 1.0): 72,
 ('পাগোল', 1.0): 1,
 ('মোন', 1.0): 2,
 ('সুদু', 1.0): 4,
 ('ভালোবাসা', 1.0): 224,
 ('পেত', 1.0): 10,
 ('চায়', 1.0): 79,
 ('কেনো', 1.0): 17,
 ('এত', 1.0): 207,
 ('ভালোবাসে', 1.0): 10,
 ('নাটক', 0.0): 2181,
 ('মজা', 0.0): 44,
 ('পাই', 0.0): 27,
 ('তাহসান', 0.0): 8,
 ('গুস্', 0.0): 1,
 ('চুদি', 0.0): 8,
 ('সুন্দর', 1.0): 1697,
 ('এক', 1.0): 1536,
 ('নাটক', 1.0): 5070,
 ('জীবনে', 0.0): 23,
 ('দেখেছি', 0.0): 8,
 ('কখনো', 0.0): 28,
 ('দেখিনি', 0.0): 11,
 ('মাঝে', 0.0): 33,
 ('জবান', 0.0): 1,
 ('বন্ধ', 0.0): 16,
 ('হয়', 0.0): 190,
 ('যায়', 0.0): 77,
 ('অসাধারণ', 1.0): 1891,
 ('অপূর্ব', 1.0): 284,
 ('ভাই', 1.0): 996,
 ('মন', 1.0): 226,
 ('ভরে', 1.0): 26,
 ('গেলো', 1.0): 58,
 ('ধন্যবাদ', 1.0): 656,
 ('পরিচালক', 1.0): 209,
 ('ভালো', 1.0): 2237,
 ('লাগলো', 1.0): 670,
 ('আবাল', 0.0): 29,
 ('

In [30]:
def extract_features(sentence, freqs):
    # process_tweet tokenizes, stems, and removes stopwords
    word_l = process_senteence(sentence)
    
    # 3 elements in the form of a 1 x 3 vector
    x = np.zeros((1, 3)) 
    
    #bias term is set to 1
    x[0,0] = 1
    for word in word_l:
        x[0,1] += freqs.get((word, 1.0),0)
        x[0,2] += freqs.get((word, 0.0),0)
    assert(x.shape == (1, 3))
    return x

In [31]:
X = np.zeros((len(x_train), 3))
for i in range(len(x_train)):
    X[i, :]= extract_features(x_train[i], freqs)

applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied third rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
appl

applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied second rules..
applied second rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
appl

applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
ap

applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied second rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied second rules..
applie

applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..

...

applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourt

applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..


applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied second rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied second rules..
applied first rules..
appl

applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied third rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applie

applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied second rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied second rules..
appli

In [32]:
from sklearn.linear_model import LogisticRegression

In [33]:
logreg = LogisticRegression()
logreg.fit(X, y_train)

X_test = np.zeros((len(x_test), 3))
for i in range(len(x_test)):
    X_test[i, :]= extract_features(x_test[i], freqs)

C:\Users\CDS\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


applied third rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
a

applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied second rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applie

applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
appl

applied fourth rules..
applied third rules..
applied second rules..
applied first rules..
applied second rules..
applied second rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied third rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
ap

...

applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules

applied first rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied first rules..
applied second rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fi

applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied third rules..
applied second rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied third rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
appl

applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied

applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied first rules..
applied second rules..
applied fourth rules..
applied second rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied third rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
appl

In [34]:
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.4f}'.format(logreg.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.8505


In [35]:
from sklearn import metrics
import numpy as np
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.77      0.72      0.74       955
         1.0       0.88      0.91      0.89      2189

    accuracy                           0.85      3144
   macro avg       0.83      0.81      0.82      3144
weighted avg       0.85      0.85      0.85      3144



In [36]:
accurecy_score = metrics.accuracy_score(y_test,y_pred)
precion_score = metrics.precision_score(y_test,y_pred)
recall_score = metrics.recall_score(y_test,y_pred)
f1_score = metrics.f1_score(y_test,y_pred)



print("Accuracy: ", accurecy_score)
print("Precision: ", precion_score)
print("Recall: ", recall_score)
print("F1 Score: ",f1_score)

Accuracy:  0.8505089058524173
Precision:  0.880141530296329
Recall:  0.9090909090909091
F1 Score:  0.8943820224719101


In [37]:
# tf idf

In [38]:
vect = TfidfVectorizer().fit(x_train)
len(vect.get_feature_names())

2971

In [39]:
x_train_vectorized = vect.transform(x_train)

In [40]:
logreg = LogisticRegression()
logreg.fit(x_train_vectorized, y_train)

C:\Users\CDS\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [41]:
x_test_vectorized = vect.transform(x_test)

In [42]:
y_pred = logreg.predict(x_test_vectorized)
print('Accuracy of logistic regression classifier on test set: {:.4f}'.format(logreg.score(x_test_vectorized, y_test)))

Accuracy of logistic regression classifier on test set: 0.8540


In [43]:
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.83      0.65      0.73       955
         1.0       0.86      0.94      0.90      2189

    accuracy                           0.85      3144
   macro avg       0.85      0.80      0.82      3144
weighted avg       0.85      0.85      0.85      3144



In [44]:
accurecy_score = metrics.accuracy_score(y_test,y_pred)
precion_score = metrics.precision_score(y_test,y_pred)
recall_score = metrics.recall_score(y_test,y_pred)
f1_score = metrics.f1_score(y_test,y_pred)



print("Accuracy: ", accurecy_score)
print("Precision: ", precion_score)
print("Recall: ", recall_score)
print("F1 Score: ",f1_score)

Accuracy:  0.8540076335877863
Precision:  0.8607172643869891
Recall:  0.9428962996802193
F1 Score:  0.8999345977763245


In [45]:
# count vectorizer
count_vector = CountVectorizer()
x_train_cv = count_vector.fit_transform(x_train)

In [46]:
# count_vector.get_feature_names()

In [47]:
x_test_cv = count_vector.transform(x_test)

In [48]:
logreg = LogisticRegression(max_iter = 10000)
logreg.fit(x_train_cv, y_train)

C:\Users\CDS\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [49]:
y_pred = logreg.predict(x_test_cv)
print('Accuracy of logistic regression classifier on test set: {:.4f}'.format(logreg.score(x_test_cv, y_test)))

Accuracy of logistic regression classifier on test set: 0.8451


In [50]:
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.83      0.62      0.71       955
         1.0       0.85      0.95      0.89      2189

    accuracy                           0.85      3144
   macro avg       0.84      0.78      0.80      3144
weighted avg       0.84      0.85      0.84      3144



In [51]:
accurecy_score = metrics.accuracy_score(y_test,y_pred)
precion_score = metrics.precision_score(y_test,y_pred)
recall_score = metrics.recall_score(y_test,y_pred)
f1_score = metrics.f1_score(y_test,y_pred)



print("Accuracy: ", accurecy_score)
print("Precision: ", precion_score)
print("Recall: ", recall_score)
print("F1 Score: ",f1_score)

Accuracy:  0.8451017811704835
Precision:  0.8493431855500821
Recall:  0.9451804476930105
F1 Score:  0.8947027027027027
